In [9]:
import time

import torch
import torch.multiprocessing as mp


def f(q):
    y = q.get()
    y[0] = 1000


def g(q):
    x = torch.zeros(1).cuda()
    x.share_memory_()
    q.put(x)
    q.put(x)
    while True:
        time.sleep(1)  # this process must live as long as x is in use

In [2]:
queue = mp.Queue()
pf = mp.Process(target=f, args=(queue,), daemon=True)
pf.start()
pg = mp.Process(target=g, args=(queue,), daemon=True)
pg.start()
pf.join()
x = queue.get()
print("x =", x.item())  # Prints x = 1000.0

x = 1000.0


Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/usr/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_32299/1644485645.py", line 18, in g
    time.sleep(1)  # this process must live as long as x is in use
KeyboardInterrupt


In [7]:
queue.put("text")

In [8]:
queue.get()

'text'

In [11]:
queue.

True

In [12]:
request_queue = mp.Queue()
response_queue = mp.Queue()

In [3]:
import torch
import torch.multiprocessing as mp


def put_text_into_q(queue, event, psend_pipe, wait_time=0.05):
    """
    Reader process, if queue is not full it will read an `ext` image from
    `images_path` and put it onto the `queue` after applying the `transform`, 
    else it will wait for `wait_time` for the  queue to free up.
    
    It uses `send_pipe` to signal downstream processes when all images have 
    been entered into the queue.

    It uses `psend_pipe` for indication.
    """
    test_text = "This is how true happiness looks like 👍😜"
    
    queue.put(test_text)
    psend_pipe.send((1, "test text benn sent"))
    
    event.set()
    queue.join()
    
    
def process_func(queue, out_queue, event, model, device, model_id):
    """
    Detector process, Reads a transformed image from the `queue`
    passes it to the detector from `get_detector` and processes the 
    output using `lock`  and `output_path` file for handling the output.
    Uses `pipe` to know if all the images have been written to
    the `queue`.
    """
    model.eval().to(device)
    while not (event.is_set() and queue.empty()):
        try:
            text = queue.get(block=True, timeout=0.1)
        except Empty:
            continue

        out = model(text)
        out_queue.put(out)
        
        if model_id == 4:
            queue.task_done()


In [15]:


def caller(device, models):
    start = time.time()
    # Initialize sync structures
    queue = mp.JoinableQueue()
    out_queue = mp.JoinableQueue()
    
    event = mp.Event()
    precv_pipe, psend_pipe = mp.Pipe(duplex=False)
    closables = [queue, precv_pipe, psend_pipe]
    lock = mp.Lock()

    # Initialize processes
    put_process = mp.Process(
        target=put_text_into_q,
        args=(queue, event, psend_pipe)
    )
    classification_processes = [
            mp.Process(
                target=process_func,
                args=(queue, out_queue, event, models[i], device, i))
            for i in range(len(models))]

    # Starting processes
    put_process.start()
    [cp.start() for cp in classification_processes]

#     print_qsize(event, precv_pipe, queue)

    # Waiting for processes to complete
    [cp.join() for cp in classification_processes]
    put_process.join()

    # Closing everything
    [c.close() for c in closables]
    print(f"time taken : {time.time() - start} s.")

In [5]:
mp.set_start_method("spawn")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from typing import List
from configs.config import AppConfig, ModelConfig

from infrastructure.models import TransformerTextClassificationModel


def build_models(model_configs: List[ModelConfig]) -> List[TransformerTextClassificationModel]:
    models = [
            TransformerTextClassificationModel(conf.model, conf.model_path, conf.tokenizer)
            for conf in model_configs
        ]
    return models


config = AppConfig.parse_file("./configs/app_config.yaml")
models = build_models(config.models)

/transformers/src/transformers/modeling_utils.py:429: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(checkpoint_file, framework="pt") as f:
/usr/local/lib/python3.8/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
/usr/local/lib/python3.8/dist-packages/torch/storage.py:899: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to y

In [16]:
caller(device, models)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'put_text_into_q' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/lib/python3.8/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'process_func' on <module '__main__' (built-in)>


KeyboardInterrupt: 